In [1]:
import pandas as pd
import numpy as np
from sklearn.base import clone
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, balanced_accuracy_score

from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
# Fix/initiate randomness
seed = 42
rs = np.random.RandomState(np.random.MT19937(np.random.SeedSequence(seed)))

Create a (binary) classification data set which has 1000 samples and 100 features. Out of the 100 features
* 1 really contains information
* `class_sep` which indicates how easy the classification task will be (larger numbers -> easier task) set to 0.2

In [3]:
n_samples, n_features, n_classes = 1_000, 100, 2
X, y = make_classification(n_samples=n_samples, n_features=n_features, n_classes=n_classes, n_informative=1, n_redundant=0, n_repeated=0, class_sep=0.2, n_clusters_per_class=1, random_state=rs)

In [4]:
np.unique(y, return_counts=True)

(array([0, 1]), array([499, 501]))

In [5]:
# Various helper functions 

def get_models(random_state):
    return LogisticRegression(penalty="none"), RandomForestClassifier(random_state=random_state)


def report_performance(model, X, y, cv=None):
    model_name = model.__class__.__name__
    if cv is not None:
        AUC_mean = cv.test_roc_auc.mean()
        AUC_std = cv.test_roc_auc.std()
        ACC_mean = cv.test_balanced_accuracy.mean()
        ACC_std = cv.test_balanced_accuracy.std()
        perf_string = f"AUC={AUC_mean:.4f} ({AUC_std:.4f}); ACC={ACC_mean:.4f} ({ACC_std:.4f})"
    else:
        AUC = roc_auc_score(y_true=y, y_score=model.predict_proba(X)[:, 1])
        ACC = balanced_accuracy_score(y_true=y, y_pred=model.predict(X))
        perf_string = f"AUC={AUC:.4f}; ACC={ACC:.4f}"

    print(f"{model_name}: " + perf_string)

For the following experiments we will create two models:
1. A standard linear Logistic Regression (with no penalty)
2. A standard non-linear model: Random Forest classifier

In [6]:
linear_model, nonlinear_model = get_models(rs)

### 1. Experiment: Overfitting

#### 1.1 Wrong way: train + test on the same data

In [7]:
linear_model.fit(X, y)
nonlinear_model.fit(X, y)

print("Train+Test on same data = overfitting:")
report_performance(linear_model, X, y)
report_performance(nonlinear_model, X, y)

Train+Test on same data = overfitting:
LogisticRegression: AUC=0.7320; ACC=0.6590
RandomForestClassifier: AUC=1.0000; ACC=1.0000


#### 1.2 Right way: train + test using cross-validation

In [8]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

cv_scores_lin = cross_validate(clone(linear_model), X, y, cv=cv, scoring=["roc_auc", "balanced_accuracy"], n_jobs=2)
cv_scores_nlin = cross_validate(clone(nonlinear_model), X, y, cv=cv, scoring=["roc_auc", "balanced_accuracy"], n_jobs=2)

print("Train+Test via cross-validation = good:")
report_performance(linear_model, X, y, cv=pd.DataFrame(cv_scores_lin))
report_performance(nonlinear_model, X, y, cv=pd.DataFrame(cv_scores_nlin))

Train+Test via cross-validation = good:
LogisticRegression: AUC=0.5982 (0.0491); ACC=0.5659 (0.0430)
RandomForestClassifier: AUC=0.6017 (0.0502); ACC=0.5672 (0.0475)


*Summary*:
- Nothing magical here: if you fit and validate your model on the same data set you will overfit
- Non-linear models can overfit more easily

### 2. Experiment: Data Leakage during Feature Selection

In [9]:
from sklearn.feature_selection import SelectPercentile

#### 2.1 Wrong way: Perform feature selection on the whole data set and then use cross-validation to fit/validate the model

In [10]:
feature_selection = SelectPercentile(percentile=10)
X_sel = feature_selection.fit_transform(X, y)

cv_scores_lin = cross_validate(clone(linear_model), X_sel, y, cv=cv, scoring=["roc_auc", "balanced_accuracy"], n_jobs=2)
cv_scores_nlin = cross_validate(clone(nonlinear_model), X_sel, y, cv=cv, scoring=["roc_auc", "balanced_accuracy"], n_jobs=2)

print("Feature selection on all data + cross-validation afterwards = bias:")
report_performance(linear_model, X, y, cv=pd.DataFrame(cv_scores_lin))
report_performance(nonlinear_model, X, y, cv=pd.DataFrame(cv_scores_nlin))


Feature selection on all data + cross-validation afterwards = bias:
LogisticRegression: AUC=0.6798 (0.0518); ACC=0.6250 (0.0420)
RandomForestClassifier: AUC=0.6661 (0.0398); ACC=0.6251 (0.0325)


#### 2.2 Right way: Feature selection + model fit only on the training set, validation of the whole pipeline on the independent test set

In [11]:
from sklearn.pipeline import make_pipeline

linear_pipeline = make_pipeline(clone(feature_selection), clone(linear_model))
nonlinear_pipeline = make_pipeline(clone(feature_selection), clone(nonlinear_model))

cv_scores_lin = cross_validate(linear_pipeline, X, y, cv=cv, scoring=["roc_auc", "balanced_accuracy"], n_jobs=2)
cv_scores_nlin = cross_validate(nonlinear_pipeline, X, y, cv=cv, scoring=["roc_auc", "balanced_accuracy"], n_jobs=2)

print("Feature selection + modeling via cross-validation = good:")
report_performance(linear_pipeline, X, y, cv=pd.DataFrame(cv_scores_lin))
report_performance(nonlinear_pipeline, X, y, cv=pd.DataFrame(cv_scores_nlin))


Feature selection + modeling via cross-validation = good:
Pipeline: AUC=0.6348 (0.0480); ACC=0.5960 (0.0460)
Pipeline: AUC=0.6188 (0.0428); ACC=0.5941 (0.0407)


*Summary*:
- If you have data leakage (i.e. using part/all of your test set for preprocessing) you will inflate your performance estimate

However, the bias seems to be very small. Why?

#### 2.3 More extreme bias: p >> n case

Create a new classification data set where the number of features is larger than number of samples

In [12]:
# p > n regime:
n_samples_new, n_features_new = 100, 1_000
X_new, y_new = make_classification(n_samples=n_samples_new, n_features=n_features_new, n_classes=n_classes, n_informative=1, n_redundant=0, class_sep=0.2, n_clusters_per_class=1, random_state=rs)

#### 2.3.1 Wrong way

In [13]:
feature_selection = SelectPercentile(percentile=10)
X_new_sel = feature_selection.fit_transform(X_new, y_new)

cv_scores_lin = cross_validate(clone(linear_model), X_new_sel, y_new, cv=cv, scoring=["roc_auc", "balanced_accuracy"], n_jobs=2)
cv_scores_nlin = cross_validate(clone(nonlinear_model), X_new_sel, y_new, cv=cv, scoring=["roc_auc", "balanced_accuracy"], n_jobs=2)

print("p >> n:")
print("Feature selection on all data + cross-validation afterwards = bias:")
report_performance(linear_model, X, y, cv=pd.DataFrame(cv_scores_lin))
report_performance(nonlinear_model, X, y, cv=pd.DataFrame(cv_scores_nlin))

p >> n:
Feature selection on all data + cross-validation afterwards = bias:
LogisticRegression: AUC=0.9760 (0.0506); ACC=0.9500 (0.0527)
RandomForestClassifier: AUC=0.9380 (0.0802); ACC=0.8800 (0.1033)


#### 2.3.2 Right way

In [14]:
cv_scores = cross_validate(clone(linear_pipeline), X_new, y_new, cv=cv, scoring=["roc_auc", "balanced_accuracy"], n_jobs=2)
cv_scores = cross_validate(clone(nonlinear_pipeline), X_new, y_new, cv=cv, scoring=["roc_auc", "balanced_accuracy"], n_jobs=2)

print("p >> n")
print("Feature selection + modeling via cross-validation = good:")
report_performance(linear_model, X, y, cv=pd.DataFrame(cv_scores))
report_performance(nonlinear_model, X, y, cv=pd.DataFrame(cv_scores))


p >> n
Feature selection + modeling via cross-validation = good:
LogisticRegression: AUC=0.5820 (0.1922); ACC=0.5200 (0.1751)
RandomForestClassifier: AUC=0.5820 (0.1922); ACC=0.5200 (0.1751)


*Summary*:
- The bias is indeed much larger in the p >> n case (for both linear and non-linear models)
- The increased sample size protects against the incorrect validation

### 3. Experiment: Improper validation while using cross-validation: the case for ~~nasty~~ nested cross-validation 

In [15]:
from sklearn.feature_selection import RFECV

#### 3.1 Wrong way: Performing a selection of optimal number of features via cross-validation __and__ using the obtained cross-validated values as reported performance

In [16]:
rfe_linear = RFECV(estimator=clone(linear_model), cv=cv, scoring="roc_auc", n_jobs=4)
rfe_nonlinear = RFECV(estimator=clone(nonlinear_model), cv=cv, scoring="roc_auc", n_jobs=4)

rfe_linear.fit(X, y)
rfe_nonlinear.fit(X, y)

print("RFE-CV selection used as reporting:")
print(f"{linear_model.__class__.__name__}: AUC={rfe_linear.cv_results_['mean_test_score'][0]:.4f} ({rfe_linear.cv_results_['std_test_score'][0]:.4f})")
print(f"{nonlinear_model.__class__.__name__}: AUC={rfe_nonlinear.cv_results_['mean_test_score'][0]:.4f} ({rfe_nonlinear.cv_results_['std_test_score'][0]:.4f})")

RFE-CV selection used as reporting:
LogisticRegression: AUC=0.6638 (0.0568)
RandomForestClassifier: AUC=0.5777 (0.0445)


#### 3.2 Right (but computational intensive way): nested cross-validation

![nested-CV](https://vitalflux.com/wp-content/uploads/2020/08/Screenshot-2020-08-30-at-6.33.47-PM.png)

In [17]:
# inner CV with 5 folds
cv_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

rfe_linear = RFECV(estimator=clone(linear_model), cv=cv_inner, scoring="roc_auc", n_jobs=4)
rfe_nonlinear = RFECV(estimator=clone(nonlinear_model), cv=cv_inner, scoring="roc_auc", n_jobs=4)

cv_scores_lin = cross_validate(rfe_linear, X, y, cv=cv, scoring=["roc_auc", "balanced_accuracy"])
cv_scores_nlin = cross_validate(rfe_nonlinear, X, y, cv=cv, scoring=["roc_auc", "balanced_accuracy"])

print("Nested cross-validation:")
report_performance(linear_model, X, y, cv=pd.DataFrame(cv_scores_lin))
report_performance(nonlinear_model, X, y, cv=pd.DataFrame(cv_scores_nlin))

Nested cross-validation:
LogisticRegression: AUC=0.6638 (0.0598); ACC=0.6201 (0.0463)
RandomForestClassifier: AUC=0.6122 (0.0347); ACC=0.5862 (0.0274)


*Summary*:
- While nested CV is the correct way of performing feature selection + validation their does not seem to be any bias
- Again the large size of the data set *protects* the validation pipeline
- The observed difference might come from the fact that I used 10-fold CV for the outer CV and 5-fold CV for the inner CV and therefore the two experiments don't match

#### 3.3 Hyperameter tuning + performance reporting

##### 3.3.1 Wrong way: grid-search via CV on the data set used as reporting metric

In [18]:
from sklearn.model_selection import GridSearchCV

In [19]:
# Should use LogisticRegressionCV instead but want to keep it the same
linear_model = LogisticRegression(penalty="l2")
gs_linear_model = GridSearchCV(estimator=linear_model, param_grid={"C": [0.01, 0.1, 1, 10, 100]}, cv=cv, scoring="roc_auc", n_jobs=4)
gs_nonlinear_model = GridSearchCV(estimator=clone(nonlinear_model), param_grid={"max_features": ["sqrt", "log2", 0.2, 0.4, 0.6]}, cv=cv, scoring="roc_auc", n_jobs=4)

gs_linear_model.fit(X, y)
gs_nonlinear_model.fit(X, y)

print("Grid search via CV and reporting of the same performance:")
print(f"{linear_model.__class__.__name__}: Best-AUC: {gs_linear_model.best_score_:.4f}")
print(f"{nonlinear_model.__class__.__name__}: Best-AUC: {gs_nonlinear_model.best_score_:.4f}")

Grid search via CV and reporting of the same performance:
LogisticRegression: Best-AUC: 0.5982
RandomForestClassifier: Best-AUC: 0.6212


##### 3.3.2 Correct way: nested-CV

In [20]:
gs_linear_model = GridSearchCV(estimator=linear_model, param_grid={"C": [0.01, 0.1, 1, 10, 100]}, cv=cv_inner, scoring="roc_auc", n_jobs=4)
gs_nonlinear_model = GridSearchCV(estimator=clone(nonlinear_model), param_grid={"max_features": ["sqrt", "log2", 0.2, 0.4, 0.6]}, cv=cv_inner, scoring="roc_auc", n_jobs=4)

cv_scores_lin = cross_validate(gs_linear_model, X, y, cv=cv, scoring=["roc_auc", "balanced_accuracy"])
cv_scores_nlin = cross_validate(gs_nonlinear_model, X, y, cv=cv, scoring=["roc_auc", "balanced_accuracy"])

print("Grid-search + performance reporting via nested cross-validation:")
report_performance(linear_model, X, y, cv=pd.DataFrame(cv_scores_lin))
report_performance(nonlinear_model, X, y, cv=pd.DataFrame(cv_scores_nlin))

Grid-search + performance reporting via nested cross-validation:
LogisticRegression: AUC=0.5982 (0.0491); ACC=0.5659 (0.0430)
RandomForestClassifier: AUC=0.6206 (0.0600); ACC=0.5972 (0.0429)


### 3.4 Overall Results (AUC):

|                    | Overfitting |    CV    | FS-wrong | FS-right | RFE-CV | RFE-nested-CV | GS-CV | GS-nested-CV |
|--------------------|:-----------:|:--------:|:--------:|:--------:|:------:|:-------------:|:-----:|:------------:|
|Logistic Regression |   0.7320    |  0.5982  |  0.6798  |  0.6348  | 0.6638 |    0.6638     | 0.5982| 0.5982       |
|Random Forest       |   1.0       |  0.6017  |  0.6661  |  0.6188  | 0.5777 |    0.6122     | 0.6212| 0.6206       |


### 3.5 Summary

- Proper validation requires nested CV
- Experiments show here that this might not be relevant in the case of n >> p data sets as the large number of samples _protects_ against the potential of overfitting 
- Bias _might_ be compounded when combining all the steps in a non-nested way
- Bias is always huge if p >> n